## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

In [ ]:
from dotenv import load_dotenv

load_dotenv('./../.env')

### Sequential LCEL Chain

In [11]:
from langchain_ollama import ChatOllama

from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate
                                        )

from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(base_url=base_url, model=model)

system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]

template = ChatPromptTemplate(messages)

In [12]:
# create chain
# input | model | output (str output)
chain = template | llm | StrOutputParser()

In [ ]:
response = chain.invoke({'school': 'elementary', 'topics': 'sun', 'points': 5})
print(response)

### Chaining Runnables (Chain Multiple Runnables)

- We can even combine this chain with more runnables to create another chain.
- Let's see how easy our generated output is?

In [ ]:
analysis_prompt = ChatPromptTemplate.from_template('analyze the following fact: {fact}\n\nHow easy is this fact to understand? Tell me in 10 words.')
analysis_prompt

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'fact': response})
print(output)

In [ ]:
composed_chain = {"fact": chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'school': 'ph.d', 'topics': 'sun', 'points': 5})
print(output)

### Parallel LCEL Chain
- Parallel chains are used to run multiple runnables in parallel.
- The final return value is a dict with the results of each value under its appropriate key.

In [28]:
llm = ChatOllama(base_url=base_url, model=model)

system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')

messages = [system, question]

template = ChatPromptTemplate(messages)

fact_chain = template | llm | StrOutputParser()

In [29]:
question = HumanMessagePromptTemplate.from_template('write a poem about the {topics} in {sentences} sentences')

messages = [system, question]

template = ChatPromptTemplate(messages)

poem_chain = template | llm | StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(fact = fact_chain, poem=poem_chain)

response = chain.invoke({'school': 'elementary', 'topics': 'sun', 'points': 5, 'sentences': 5})
print(response)

In [ ]:
response.keys()

print(response['fact'])

print("\n\n")
print(response['poem'])

### Chain Router
- The router chain is used to route the output of a previous runnable to the next runnable based on the output of the previous runnable.

In [ ]:
prompt = """Given the user review below, classify it as either being about `Positive` or `Negative`.
            Do not respond with more than one word.

            Question: {question}
            Classification:"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

question = "Thank you so much for providing such a great platform for learning."
chain.invoke({"question": question})

In [87]:
positive_prompt = """
                You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Question: {question}
                Answer:"""
positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()


negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'udemy@kgptalkie.com'.
                Question: {question}
                Answer:"""
negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()




In [88]:
def route(info):
    if "positive" in info["topic"].lower():
        return positive_chain
    else:
        return negative_chain

In [89]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)

In [ ]:
question

In [ ]:
response = full_chain.invoke({"question": question})
print(response)

In [ ]:
question = "I am not happy with the course content. I want my refund."
response = full_chain.invoke({"question": question})
print(response)

### Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda
- This is useful for formatting or when you need functionality not provided by other LangChain components, and custom functions used as Runnables are called RunnableLambdas.



In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())


prompt = ChatPromptTemplate.from_template("what is {in1} + {in2}")

chain1 = prompt | llm

chain = (
    prompt
    | llm
    | StrOutputParser()
    |{
        "char_counts": RunnableLambda(char_counts),
        "word_counts": RunnableLambda(word_counts),
        "output": RunnablePassthrough()
    }
)

chain.invoke({"in1": "bar", "in2": "gah"})

### Custom Chain using `@chain` decorator

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain


@chain
def custom_chain(params):
    return {
        "facts": fact_chain.invoke(params),
        "poem": poem_chain.invoke(params)
    }


params = {'school': 'elementary', 'topics': 'sun', 'points': 5, 'sentences': 5}
custom_chain.invoke(params)

In [ ]:
fact_chain

In [ ]:
poem_chain